In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [ ]:
# RFSoC experiment configuration
# +------------------------------------------------------------------------------------------------+
# | Feature: Single Channel 312.5 MHz BW Tx/Rx Experiment                                          |
# +-----------------+--------------------+----------------+-----------+---------------+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO           | Ref. Clock |
# | 4.0 GSPS        | 40x                | 100 MSPS       | 100M      | 1000:1800:200M| 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+---------------+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO           | Ref. Clock |
# | 2.0 GSPS        | 20x                | 100 MSPS       | 100M      | 1000:1800:200M| 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+---------------+------------+

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v39_100M.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 4000.0,
    "DACInterpolationRate": 40,
    "DACNCO": 0.0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 20,
    "ADCNCO": -0.0,
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
import numpy as np

for inv_sinc in [0, 1]:
    
    for nco in np.arange(50.0, 2000.0, 50.0):
    
        # Update config
        ZCU216_CONFIG['DACNCO'] = nco
        ZCU216_CONFIG['InvSincFIR'] = inv_sinc
        ZCU216_CONFIG['CONFIG_NAME'] = "Tx_100M_BW" + "_" + str(nco) + "_" + str(inv_sinc)
        
        print(f"Experiment InvSincFIR {inv_sinc}, DAC NCO {nco}")

        rfdc_t = RfdcTask(ol)

        for task in [rfdc_t]:
            task.start()
            task.join()

        led_t = BlinkLedTask(ol)
        tx_t = SingleChTxTask(ol, tx_file="./tmp_files/Tx.mat")

        parallel_task = [led_t, tx_t]
        for task in parallel_task:
            task.start()

        dummy = input('Type anything and press Enter to continue.')

        for task in parallel_task:
            task.stop()